In [130]:
import pandas as pd
import duckdb
import string

data = (pd.read_csv('merged_gw.csv')
        .drop(columns = "kickoff_time"))

duckdb.sql('''with init as (
  
  select *, rank() over(partition by position order by tot_points desc) as rank
           
        from (
        select name, position, sum(total_points) as tot_points
        
        from data
           
        group by name, position
        order by name, position)
           
           order by tot_points desc
  )
  
  select name, position, tot_points
  
  from init
  
  where rank = 1
  and position != 'GK';
           ''')

┌──────────────────┬──────────┬────────────┐
│       name       │ position │ tot_points │
│     varchar      │ varchar  │   int128   │
├──────────────────┼──────────┼────────────┤
│ Mohamed Salah    │ MID      │         59 │
│ Ollie Watkins    │ FWD      │         59 │
│ Joachim Andersen │ DEF      │         51 │
└──────────────────┴──────────┴────────────┘

In [144]:
import pandas as pd
import duckdb
import string

data = (pd.read_csv('merged_gw.csv')
        .drop(columns = "kickoff_time"))

indicators = pd.read_json('queries.json')

with pd.ExcelWriter("FPL.xlsx", engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        
        for index, row in indicators.iterrows():
                num = 0
                for c in row["startcol"]:
                        if c in string.ascii_letters:
                                num = num * 26 + (ord(c.upper()) - ord('A'))
                
                result = (duckdb.sql(row["query"])
                          .to_df())
                
                result.to_excel(writer, index=False,
                                startcol=num,
                                startrow=row["startrow"]-1,
                                header=False)

In [110]:
for index, row in indicators.iterrows():
    result = duckdb.sql(row["query"]).to_df()
    print(result)

    

              team     1     2     3     4     5      6     7     8
0          Arsenal  42.0  58.0  36.0  53.0  65.0   31.0  89.0  61.0
1      Aston Villa  20.0  88.0  58.0  17.0  46.0   60.0  66.0  35.0
2      Bournemouth  32.0  37.0  19.0  36.0  56.0   24.0  17.0  19.0
3        Brentford  38.0  76.0  37.0  42.0  21.0   30.0  36.0  28.0
4         Brighton  58.0  59.0  31.0  52.0  55.0   47.0  13.0  36.0
5          Burnley  19.0   NaN  28.0  25.0  31.0   27.0  64.0  21.0
6          Chelsea  37.0  20.0  80.0  23.0  49.0   24.0  68.0  52.0
7   Crystal Palace  61.0  22.0  36.0  44.0  26.0   50.0  63.0  54.0
8          Everton  24.0  10.0  23.0  34.0  26.0   51.0  27.0  74.0
9           Fulham  60.0  18.0  35.0  23.0  64.0   53.0  18.0  45.0
10       Liverpool  33.0  44.0  38.0  78.0  46.0   53.0  21.0  37.0
11           Luton  27.0   NaN  19.0  32.0  25.0   37.0  67.0  25.0
12        Man City  76.0  59.0  41.0  68.0  54.0   60.0  23.0  27.0
13         Man Utd  63.0  19.0  43.0  28.0  29.0

In [145]:
for index, row in indicators.iterrows():
        num = 0
        for c in row["startcol"]:
                if c in string.ascii_letters:
                        num = num * 26 + (ord(c.upper()) - ord('A')) + 1
        
        print(num)

2
12
2
